In [1]:
!pip install -q faiss-cpu sentence-transformers requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 82.3 MB/s eta 0:00:00


In [2]:
import faiss
import numpy as np
import requests
from sentence_transformers import SentenceTransformer


In [7]:
documents = {
    "Construction_FAQ.txt": """
    What are construction project delays? Construction delays occur when a project
    takes longer than the planned timeline. Delays are primarily caused by resource
    constraints, weather conditions, equipment failures, and regulatory approvals.

    What factors affect construction project delays? Multiple factors impact scheduling:
    1. Resource Constraints: Labor shortages and material unavailability directly impact
    the timeline. Skilled labor is often scarce, and procurement of specialized materials
    can take 3-4 weeks.
    2. Weather Conditions: Seasonal weather impacts all outdoor work. Rain, extreme heat,
    and cold temperatures can delay projects by an average of 2 weeks per project cycle.
    3. Equipment Failures: Construction equipment requires regular maintenance. Breakdowns
    can halt operations for days or weeks.
    4. Regulatory Approvals: Permits and inspections must be completed before proceeding
    to the next phase. These approvals typically take 1-2 weeks.

    How long does material procurement take? Material procurement typically requires
    3-4 weeks from order placement to delivery. This includes supplier identification,
    ordering, manufacturing, and transportation time. International materials may take
    6-8 weeks.
    """,

    "Project_Policies.md": """
    Construction Project Management Policies

    Risk Management: All projects must identify potential risks during the planning phase.
    Key risks include supply chain disruptions from vendor delays or quality issues,
    equipment availability and maintenance schedules, regulatory changes that may affect
    project scope, labor availability in the construction market, and budget fluctuations
    due to material price changes.

    Scheduling Best Practices: Projects must account for buffer time for unexpected delays
    (typically 10-15% of total duration), seasonal weather patterns and construction constraints,
    resource allocation and team availability, third-party dependencies and approval processes.

    Quality Standards: All deliverables must meet industry standards and specifications.
    Regular inspections are required at key milestones. Documentation of all approvals and
    sign-offs must be maintained throughout the project lifecycle.
    """,

    "Technical_Specifications.txt": """
    TECHNICAL SPECIFICATIONS - CONSTRUCTION PROJECT EXECUTION

    Project Timeline Breakdown:
    Planning Phase: 2-4 weeks (design, permits, resource allocation)
    Material Procurement: 3-4 weeks parallel to planning
    Site Preparation: 1-2 weeks (clearing, foundation setup)
    Main Construction: 8-16 weeks depending on scope
    Quality Assurance & Testing: 2-4 weeks
    Final Inspections & Handover: 1-2 weeks

    Resource Requirements:
    Skilled Labor: 10-50 workers depending on project scale
    Equipment: Excavators, cranes, compressors based on project type
    Materials: Concrete, steel, wood, electrical components
    Machinery Maintenance: Weekly inspections recommended

    Weather Considerations:
    Rain delays: Average 2 weeks per project in monsoon regions
    Temperature extremes: Work halts below 5°C or above 40°C
    Wind conditions: Heavy equipment operation restricted above 50 km/h
    """
}


Document Chunking

In [8]:
def chunk_text(text, chunk_size=300):
    sentences = text.split(".")
    chunks, current = [], ""

    for s in sentences:
        if len(current) + len(s) < chunk_size:
            current += s + "."
        else:
            chunks.append(current.strip())
            current = s + "."

    if current.strip():
        chunks.append(current.strip())

    return chunks


 Chunks with Metadata

In [9]:
chunk_data = []

for doc_name, text in documents.items():
    chunks = chunk_text(text)
    for i, chunk in enumerate(chunks):
        chunk_data.append({
            "id": f"{doc_name}_chunk_{i}",
            "source": doc_name,
            "text": chunk
        })

texts = [c["text"] for c in chunk_data]


Embeddings

In [10]:
embedder = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedder.encode(texts)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

FAISS Vector Index

In [11]:
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings).astype("float32"))


Semantic Retrieval (Top-K)

In [12]:
def retrieve_chunks(query, top_k=2):
    query_vec = embedder.encode([query])
    distances, indices = index.search(
        np.array(query_vec).astype("float32"),
        top_k
    )

    return [chunk_data[i] for i in indices[0]]


In [13]:
OPENROUTER_API_KEY = "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"


In [27]:
def generate_answer(question, context):
    prompt = f"""
Answer ONLY using the context below.
If the answer is not explicitly stated, say:
"Not specified in the documents."

Context:
{context}

Question:
{question}
"""

    response = requests.post(
        "https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {OPENROUTER_API_KEY}",
            "Content-Type": "application/json"
        },
        json={
            "model": "mistralai/devstral-2512:free",
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0.0
        }
    )

    data = response.json()
    if "choices" not in data:
        return "Error: Invalid LLM response"

    return data["choices"][0]["message"]["content"]


Sample Questions

In [28]:
question = "What factors affect construction project delays?"
retrieved = retrieve_chunks(question)
context_text = "\n".join([c["text"] for c in retrieved])
answer = generate_answer(question, context_text)
print("Answer:", answer)


Answer: The factors that affect construction project delays include:

1. Resource Constraints: Labor shortages and material unavailability directly impact the timeline. Skilled labor is often scarce, and procurement of specialized materials can take 3-4 weeks.
2. Weather Conditions
3. Equipment Failures
4. Regulatory Approvals


In [29]:
question2 = "What resource constraints affect construction schedules?"
retrieved = retrieve_chunks(question2)
context_text = "\n".join([c["text"] for c in retrieved])
answer2 = generate_answer(question2, context_text)
print("Answer:", answer2)


Answer: Resource constraints that affect construction schedules include labor shortages and material unavailability. Skilled labor is often scarce, and procurement of specialized materials can take 3-4 weeks.


In [33]:
question3 = "How long does material procurement typically take in construction projects?"
retrieved = retrieve_chunks(question3)
context_text = "\n".join([c["text"] for c in retrieved])
answer3 = generate_answer(question3, context_text)
print("Answer:", answer3)


Answer: Material procurement typically requires 3-4 weeks from order placement to delivery.


In [31]:
question4 = "What weather conditions restrict construction work?"
retrieved = retrieve_chunks(question4)
context_text = "\n".join([c["text"] for c in retrieved])
answer4 = generate_answer(question, context_text)
print("Answer:", answer4)


Answer: The factors that affect construction project delays, as specified in the documents, are:

1. **Weather Conditions**:
   - Rain (average 2 weeks delay per project in monsoon regions).
   - Extreme temperatures (work halts below 5°C or above 40°C).
   - Wind conditions (heavy equipment operation restricted above 50 km/h).

2. **Equipment Failures**: Construction equipment requires regular maintenance, which can lead to delays if not properly managed.

Other potential factors (e.g., labor shortages, permit delays) are not explicitly mentioned in the provided context.


In [32]:
question5 = "At what wind speed is heavy equipment operation restricted?"
retrieved = retrieve_chunks(question5)
context_text = "\n".join([c["text"] for c in retrieved])
answer5 = generate_answer(question5, context_text)
print("Answer:", answer5)


Answer: Heavy equipment operation is restricted above 50 km/h.
